In [2]:
# RESET VECTOR DATABASE (MANDATORY)

import shutil

shutil.rmtree("vector_db", ignore_errors=True)
print("Vector DB reset completed.")

Vector DB reset completed.


In [3]:
!git clone https://github.com/kalpeshdahake/SecRAG-10K.git

Cloning into 'SecRAG-10K'...
remote: Enumerating objects: 143, done.
remote: Counting objects: 100% (143/143), done.
remote: Compressing objects: 100% (125/125), done.
remote: Total 143 (delta 50), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (143/143), 1.36 MiB | 13.67 MiB/s, done.
Resolving deltas: 100% (50/50), done.


In [4]:
%cd SecRAG-10K

/content/SecRAG-10K


In [5]:
!ls

data		  embeddings  llm	README.md	  retrieval
design_report.md  ingestion   pipeline	requirements.txt


In [6]:
!pip install -r requirements.txt --no-cache-dir

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.0/329.0 kB 139.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 232.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 282.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 274.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 252.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 159.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 244.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 274.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 219.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.0/220.0 kB 285.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 257.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [7]:
!ls data

10-Q4-2024-As-Filed.pdf  tsla-20231231-gen.pdf


In [8]:
from ingestion.pdf_loader import load_pdf
from ingestion.section_parser import assign_items
from ingestion.chunker import chunk_text

from embeddings.embedder import Embedder
from embeddings.vector_store import VectorStore

from pipeline.rag_pipeline import answer_question

In [9]:
embedder = Embedder()

vector_store = VectorStore(persist_dir="vector_db")
collection = vector_store.get_or_create_collection("financial_10k")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [10]:
apple_pages = load_pdf(
    "data/10-Q4-2024-As-Filed.pdf",
    company="Apple",
    document="Apple 10-K"
)

apple_pages = assign_items(apple_pages)
apple_chunks = chunk_text(apple_pages)

print("Apple pages:", len(apple_pages))
print("Apple chunks:", len(apple_chunks))
print("Sample Apple metadata:", apple_chunks[0]["metadata"])

Apple pages: 121
Apple chunks: 605
Sample Apple metadata: {'company': 'Apple', 'document': 'Apple 10-K', 'page': 1, 'item': 'Unknown'}


In [11]:
apple_embeddings = embedder.embed_texts(
    [chunk["text"] for chunk in apple_chunks]
)

vector_store.add_chunks(
    collection,
    apple_chunks,
    apple_embeddings
)

print("Apple indexed successfully")

Batches:   0%|          | 0/19 [00:00<?, ?it/s]

Apple indexed successfully


In [12]:
tesla_pages = load_pdf(
    "data/tsla-20231231-gen.pdf",
    company="Tesla",
    document="Tesla 10-K"
)

tesla_pages = assign_items(tesla_pages)
tesla_chunks = chunk_text(tesla_pages)

print("Tesla pages:", len(tesla_pages))
print("Tesla chunks:", len(tesla_chunks))
print("Sample Tesla metadata:", tesla_chunks[0]["metadata"])

Tesla pages: 130
Tesla chunks: 647
Sample Tesla metadata: {'company': 'Tesla', 'document': 'Tesla 10-K', 'page': 1, 'item': 'Unknown'}


In [13]:
tesla_embeddings = embedder.embed_texts(
    [chunk["text"] for chunk in tesla_chunks]
)

vector_store.add_chunks(
    collection,
    tesla_chunks,
    tesla_embeddings
)

print("Tesla indexed successfully")

Batches:   0%|          | 0/21 [00:00<?, ?it/s]

Tesla indexed successfully


In [14]:
# Optional quick retrieval sanity check
results = collection.query(
    query_texts=["total revenue"],
    n_results=3
)

for meta in results["metadatas"][0]:
    print(meta)

/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:08<00:00, 9.86MiB/s]


{'company': 'Tesla', 'chunk_id': 'Tesla_Tesla 10-K_p94_c520', 'item': 'Item 9A', 'document': 'Tesla 10-K', 'page': 94}
{'chunk_id': 'Tesla_Tesla 10-K_p60_c344', 'item': 'Item 9A', 'document': 'Tesla 10-K', 'company': 'Tesla', 'page': 60}
{'page': 39, 'company': 'Apple', 'chunk_id': 'Apple_Apple 10-K_p39_c200', 'document': 'Apple 10-K', 'item': 'Item 8'}


In [15]:
response = answer_question(
    "What was Apple’s total revenue for the fiscal year ended September 28, 2024?",
    collection
)

print(response["answer"])
print(response["sources"])

config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


$391,035 million
['Apple 10-K, Item 8, p. 32']


In [16]:
evaluation_questions = [
    {"question_id": 1, "question": "What was Apple’s total revenue for the fiscal year ended September 28, 2024?"},
    {"question_id": 2, "question": "How many shares of common stock were issued and outstanding as of October 18, 2024?"},
    {"question_id": 3, "question": "What is the total amount of term debt (current + non-current) reported by Apple as of September 28, 2024?"},
    {"question_id": 4, "question": "On what date was Apple’s 10-K report for 2024 signed and filed with the SEC?"},
    {"question_id": 5, "question": "Does Apple have any unresolved staff comments from the SEC as of this filing?"},

    {"question_id": 6, "question": "What was Tesla’s total revenue for the year ended December 31, 2023?"},
    {"question_id": 7, "question": "What percentage of Tesla’s total revenue in 2023 came from Automotive Sales (excluding Leasing)?"},
    {"question_id": 8, "question": "What is the primary reason Tesla states for being highly dependent on Elon Musk?"},
    {"question_id": 9, "question": "What types of vehicles does Tesla currently produce and deliver?"},
    {"question_id": 10, "question": "What is the purpose of Tesla’s lease pass-through fund arrangements?"},

    {"question_id": 11, "question": "What is Tesla’s stock price forecast for 2025?"},
    {"question_id": 12, "question": "Who is the CFO of Apple as of 2025?"},
    {"question_id": 13, "question": "What color is Tesla’s headquarters painted?"}
]

In [17]:
results = []

for q in evaluation_questions:
    response = answer_question(q["question"], collection)

    results.append({
        "question_id": q["question_id"],
        "answer": response["answer"],
        "sources": response["sources"]
    })

import json
print(json.dumps(results, indent=2))

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

[
  {
    "question_id": 1,
    "answer": "$391,035 million",
    "sources": [
      "Apple 10-K, Item 8, p. 32"
    ]
  },
  {
    "question_id": 2,
    "answer": "This question cannot be answered based on the provided documents.",
    "sources": []
  },
  {
    "question_id": 3,
    "answer": "$121,983 million",
    "sources": [
      "Apple 10-K, Item 8, p. 45"
    ]
  },
  {
    "question_id": 4,
    "answer": "November 1, 2024",
    "sources": [
      "Apple 10-K, Item 601, p. 118"
    ]
  },
  {
    "question_id": 5,
    "answer": "Not specified in the document.",
    "sources": []
  },
  {
    "question_id": 6,
    "answer": "96,773 million",
    "sources": [
      "Tesla 10-K, Item 9A, p. 51"
    ]
  },
  {
    "question_id": 7,
    "answer": "19.4%",
    "sources": [
      "Tesla 10-K, Item 9A, p. 51"
    ]
  },
  {
    "question_id": 8,
    "answer": "Not specified in the document.",
    "sources": []
  },
  {
    "question_id": 9,
    "answer": "Model S and Model X, Model 3 